# Characterizing WTSN channels

This notebooks simulates a basic WTSN system with some UEs, an access point and 
some base schedules to determine what the packet latencies and throughput are under 
different conditions.

In [ ]:
# Necessary imports
from datetime import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

from network_classes import *


## Dummy simulation

Dummy simulation that marks all packets as delivered after a small wait time. Used
to test that all classes are functioning as they should.

In [ ]:
# Create a schedule with 2 slots and 2 UEs
slot1 = Slot(0, 0, 10000, "reserved", ["UE0"])
slot2 = Slot(1, 10000, 20000, "reserved", ["UE0"])
schedule = {0: slot1, 1: slot2}
base_schedule = Schedule(0, 20000, 2, schedule)
print(base_schedule)

# Create a UE
ue = UE(0,{1: 0, 2: 1}, "central control", "Mode 1",  10)
ue.generate_packets(base_schedule, [100]*10, [1]*10)
print(ue)

# Serve the packets
ue.serve_packets(base_schedule)
print(ue)

latencies = ue.obtain_packet_latency()
print(latencies)

## Simulation of a UE with periodic slots  (Simulation 1)

- The base schedule is  
  - t0 - t1: UE 1
  - t1 - t2: UE 2
  - t2 - t3: UE 3
  - t3 - t4: UE 1 and repeat
- Packets are perfectly synchronized with the 

In [ ]:
# Parameters affecting how a packet is served
# TODO: integrate MCS usage into the UE instead of having it outside
# TODO: Create a simple CSV file of this


# TODO: Remove the 67us from this that contains backoff 
parameters = {
    "setting 0": {
        "SNR": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 591.8
    },
    "setting 1": {
        "SNR": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 1470.2
    },
    "setting 2": {
        "SNR": 20,
        "MCS": 0,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 2953.4
    },
    "setting 3": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 534.2
    },
    "setting 4": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 980.6
    },
    "setting 5": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 1715
    },
    "setting 6": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 1,
        "delivery_latency": 519.8
    },
    "setting 7": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 964,
        "aggregation": 1,
        "delivery_latency": 807.8
    },
    "setting 8": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 2464,
        "aggregation": 1,
        "delivery_latency": 1311.8
    },
    "setting 9": {
        "SNR": 20,
        "MCS": 1,
        "PER": 0,
        "payload_size": 64,
        "aggregation": 10,
        "delivery_latency": 980.6
    },
    
}



In [ ]:
setting = "setting 9"
payload_size = parameters[setting]["payload_size"]*parameters[setting]["aggregation"]
delivery_latency = parameters[setting]["delivery_latency"]
PER = parameters[setting]["PER"]
PER = 0.001
# payload_size=1000
# delivery_latency=5000

num_UEs = 3
UEs = ["UE" + str(i) for i in range(num_UEs)]
num_packets_per_ue = 130  # Number of packets per UE for the whole period
packet_sizes = [parameters[setting]["payload_size"]]*num_packets_per_ue
priorities = [1]*num_packets_per_ue

## Schedule parameters
num_slots_per_UE = 100
num_slots = num_slots_per_UE*num_UEs
start_offset = 10 # microseconds
end_time = start_offset
slot_duration = 1000 # microseconds
slots = {}


# TODO: move the knowledge of how many packets there are to this part of the code
# instead of keeping it in the UE class

# Create a schedule
start_time = start_offset
for i in range(num_slots):
    slots[i] = Slot(i, start_time, start_time + slot_duration, "reserved", [UEs[i%num_UEs]])
    start_time += slot_duration
schedule = Schedule(start_offset, start_time, num_slots, slots)

# print(schedule)


# Create UEs and packets
UEs = {}
for i in range(num_UEs):
    UE_temp = UE(i, {1: 0, 2: 1}, "central control", "Mode 2",  num_packets_per_ue)
    UE_temp.generate_packets(schedule, packet_sizes, priorities)
    UEs[i] = UE_temp

# for i in UEs:
#     print(UEs[i])

# Serve the packets
for i in UEs:
    UEs[i].serve_packets(schedule, payload_size=payload_size, delivery_latency=delivery_latency,
                         PER=PER)

# print(UEs[0])

latencies = UEs[0].obtain_packet_latency()
print(latencies)
latencies = [latency for latency in latencies if latency is not None]

# TODO: Make this more general i.e handle packet statuses directly instead of opearting under the 
# restrictions of this simulation
num_packets_queued = num_packets_per_ue - len(latencies)
print(f"Number of packets not served: {num_packets_queued}")

# Plot a cdf of the latencies using the latencies variable above
latencies = np.array(latencies)
latencies = latencies/1000 # convert to milliseconds
latencies = np.sort(latencies)
yvals = (np.arange(len(latencies)) + 1)/float(len(latencies))
plt.plot(latencies, yvals)
plt.xlabel("Latency (ms)")
plt.ylabel("CDF")
title = (f"Simulation 1 \nUE_n_packets: {num_packets_per_ue}, "
         f"num_UEs: {num_UEs},"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, "
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, "
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
plt.show()

## Simulating UEs with simple poisson arrival processes

- TODO: Select packet sizes correctly
- TODO: Select slot sizes correctly
- TODO: Select PER correctly i.e modify the settings to have 10-3 PER instead of 
extrapolating
- TODO: Change the latency-subtract the 67 us from the values to have it reflect 
the TSN case

- Experiment-
  - Determine a base schedule
  - for a range of lambda values:
    - Generate packets according to the base schedule for a UE
    - Serve the packets
    - Track n_packets_not_served, latencies of packets served, number of packets generated
    - 99 percentile and average latency
  - plot latency vs lambda
  - Change base schedule repeat

TODO: there should be some notion of the base schedule itself

In [ ]:
results_directory_simulation = "./results/simulation_2/"

setting = "setting 9"
payload_size = parameters[setting]["payload_size"]*parameters[setting]["aggregation"]
delivery_latency = parameters[setting]["delivery_latency"]
PER = parameters[setting]["PER"]
# PER = 0.1
# payload_size=1000
# delivery_latency=5000

num_UEs = 3
UEs = ["UE" + str(i) for i in range(num_UEs)]
num_packets_per_ue = None  # Number of packets per UE for the whole period
packet_sizes = [parameters[setting]["payload_size"]]
priorities = [1]
lambda_range = np.logspace(-4, -2, 30)
UE_arrival = ["Poisson"]*num_UEs
UE_serve_mode = ["Mode 2"]*num_UEs


## Schedule parameters
num_slots_per_UE = 1000
num_slots = num_slots_per_UE*num_UEs
start_offset = 10 # microseconds
end_time = start_offset
slot_duration = 4000 # microseconds
slots = {}

# Plot information
percentile_to_plot = 99


In [ ]:

# TODO: move the knowledge of how many packets there are to this part of the code
# instead of keeping it in the UE class

# Create a schedule
start_time = start_offset
for i in range(num_slots):
    slots[i] = Slot(i, start_time, start_time + slot_duration, "reserved", [UEs[i%num_UEs]])
    start_time += slot_duration
schedule = Schedule(start_offset, start_time, num_slots, slots)

end_time = start_time # Due to variable use while saving experiment parameters

print(schedule)

results_per_lambda = {}

for lambda_value in lambda_range:
# Create UEs and packets
    UEs = {}
    result = {}
    # Creating a single UE is done to reduce time consumed as the behvaiour of other UEs does not
    # affect the results of the simulation. 1 can be changed to num_UEs to simulate more UEs
    for i in range(1): 
        # TODO: Move the UE creation parameters to the cell above?
        UE_temp = UE(i, {1: 0, 2: 1}, UE_arrival[i], UE_serve_mode[i],  num_packets_per_ue)
        UE_temp.set_poisson_lambda(lambda_value)
        UE_temp.generate_packets(schedule, packet_sizes, priorities)
        UEs[i] = UE_temp

    # for i in UEs:
    #     print(UEs[i])

    # Serve the packets
    for i in range(1):
        UEs[i].serve_packets(schedule, payload_size=payload_size, delivery_latency=delivery_latency,
                            PER=PER)

    # for i in UEs:
    #     print(UEs[i])

    result["latencies"] = UEs[0].obtain_packet_latency()
    result["latencies_non_zero"] = [latency for latency in result["latencies"] if latency is not None]
    result["num_packets_generated"] = UEs[0].n_packets
    result["num_packets_not_served"] = result["num_packets_generated"] - len(result["latencies_non_zero"])
    result["percentile_latency"] = compute_percentile(result["latencies_non_zero"], percentile_to_plot)
    result["mean_latency"] = np.mean(result["latencies_non_zero"])
    results_per_lambda[lambda_value] = result

    # TODO: Check that the delivery times are always in ascending order
    # TODO: check that the arrival times are always in ascending order

    # TODO: Make this more general i.e handle packet statuses directly instead of opearting under the 
    # restrictions of this simulation
    print("Num packets: " + str(UEs[0].n_packets))
    print(f"Number of packets not served: {result['num_packets_not_served']}")



In [ ]:
n_packets_generated = [results_per_lambda[lambda_value]["num_packets_generated"] \
                       for lambda_value in results_per_lambda]

plt.plot(lambda_range, n_packets_generated)

In [ ]:
# Save the parameters and the results of the experiment to a file
class NumpyEncoder(json.JSONEncoder):
    """Custom encoder for numpy data types"""
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


# Create a results directory folder using results_directory_simulation and the current time
experiment_folder_name = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
results_directory_experiment = os.path.join(results_directory_simulation, experiment_folder_name)
os.makedirs(results_directory_experiment, exist_ok=True)

experiment_parameters = {
    "setting": parameters[setting],
    "num_UEs": num_UEs,
    "num_packets_per_ue": num_packets_per_ue,
    "packet_sizes": packet_sizes,
    "priorities": priorities,
    "num_slots_per_UE": num_slots_per_UE,
    "num_slots": num_slots,
    "start_offset": start_offset, # microseconds
    "end_time": end_time,
    "slot_duration": slot_duration,
    "percentile_to_plot": percentile_to_plot,
    "lambda_range": lambda_range,
}

# Write experiment_parameters_json to a json file with filename experiment_parameters.json

experiment_parameters_json = json.dumps(experiment_parameters, indent=4, cls=NumpyEncoder)
experiment_parameters_json_filename = os.path.join(results_directory_experiment, \
                                                   "experiment_parameters.json")
with open(experiment_parameters_json_filename, "w") as file:
    file.write(experiment_parameters_json)


experiment_parameters_pickle = {
    "schedule": schedule,
    "UEs": UEs,
    "results_per_lambda": results_per_lambda,
    "experiment_parameters": experiment_parameters
}

experiment_parameters_pickle_filename = os.path.join(results_directory_experiment, \
                                                    "experiment_parameters.pkl")

with open(experiment_parameters_pickle_filename, "wb") as file:
    pickle.dump(experiment_parameters_pickle, file)

In [ ]:
# Plot cdf of latencies 
plt.figure()
for lambda_value in lambda_range:
    latencies = results_per_lambda[lambda_value]["latencies_non_zero"]
    latencies = np.array(latencies)
    latencies = latencies/1000 # convert to milliseconds
    latencies = np.sort(latencies)
    yvals = (np.arange(len(latencies)) + 1)/float(len(latencies))
    plt.plot(latencies, yvals, label=f"lambda: {lambda_value}")
plt.xlabel("Latency (ms)")
plt.ylabel("CDF")
title = (f"Simulation 2 Latency vs lambda, \n PER = {PER},"
         f"num_UEs: {num_UEs},"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, "
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, "
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
plt.savefig(os.path.join(results_directory_experiment, "latency_cdf.png"))
plt.show()


# Plot the percentile curve

plt.figure()
percentiles = []
for lambda_value in lambda_range:
    percentiles.append(results_per_lambda[lambda_value]["percentile_latency"])
plt.plot(np.array(lambda_range)*(schedule.end_time - schedule.start_time), percentiles)
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda")
plt.ylabel(str(percentile_to_plot) + "percentile latency (us)")

title = (f"Simulation 2 {percentile_to_plot} percentile latency vs lambda, \n PER = {PER},"
         f"num_UEs: {num_UEs},"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, "
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, "
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label
plt.text(0, percentiles[0], str(percentiles[0]), fontsize=12, verticalalignment='bottom')
plt.savefig(os.path.join(results_directory_experiment, "percentile_latency.png"))
plt.show()

print(schedule.end_time - schedule.start_time)

# Plot the mean latency curve

plt.figure()
mean_latencies = []
for lambda_value in lambda_range:
    mean_latencies.append(results_per_lambda[lambda_value]["mean_latency"])
plt.plot(lambda_range, mean_latencies)
plt.xlabel("lambda")
plt.ylabel("Mean latency (us)")

title = (f"Simulation 2 mean latency vs lambda, \n PER = {PER},"
         f"num_UEs: {num_UEs},"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, "
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, "
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
plt.savefig(os.path.join(results_directory_experiment, "mean_latency.png"))
plt.show()

# Plot the number of packets not served

plt.figure()
n_packets_not_served = []
for lambda_value in lambda_range:
    n_packets_not_served.append(results_per_lambda[lambda_value]["num_packets_not_served"])
plt.plot(lambda_range, n_packets_not_served)
plt.xlabel("lambda")
plt.ylabel("Number of packets not served")

title = (f"Simulation 2 Number of packets queued vs lambda, \n PER = {PER},"
         f"num_UEs: {num_UEs},"
         f"allowed_payload: {payload_size} B, \n "
         f"packet size: {packet_sizes[0]} B, "
         f"delivery_latency: {delivery_latency} us ,\n"
         f"num_slots: {num_slots}, "
         f"slot_duration: {slot_duration} us ,\n")
plt.title(title)
plt.savefig(os.path.join(results_directory_experiment, "n_packets_not_served.png"))
plt.show()

In [ ]:
arrival_times = []
for packet in UEs[0].packets:
    arrival_times.append(packet.arrival_time)

# plot histogram of arrival times
plt.hist(arrival_times, bins=100)
plt.xlabel("Arrival time (us)")
plt.ylabel("Frequency")
plt.title("Arrival time histogram")
plt.show()

arrival_times = np.array(arrival_times)
inter_arrival_times = np.diff(arrival_times)
plt.hist(inter_arrival_times, bins=100)
plt.xlabel("Inter-arrival time (us)")
plt.ylabel("Frequency")
plt.title("Inter-arrival time histogram")
plt.show()

In [ ]:
a = [12,34,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
np.percentile(a, 99)